<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Colab%205%20-%20Construir%20una%20Red%20Neuronal%20Recurrente%20en%20TensorFlow%202.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalar las dependencias y la configuración del notebook en GPU

In [5]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

## Paso 2: Importar las librerías necesarias

In [6]:
import tensorflow as tf

from tensorflow.keras.datasets import imdb

In [7]:
tf.__version__

'2.4.1'

## Paso 3: Pre procesado de datos


### Configurar parámetros del dataset

In [8]:
number_of_words = 20000
max_len = 100

### Carga del dataset de IMDB

link donde esta el dataset https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews      
IMDB dataset having 50K movie reviews for natural language processing or Text analytics.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training and 25,000 for testing. So, predict the number of positive and negative reviews using either classification or deep learning algorithms.
For more dataset information, please go through the following link,
http://ai.stanford.edu/~amaas/data/sentiment/

In [9]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

### Cortar secuencias de texto de la misma longitud

In [11]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)

In [12]:
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

### Configurar parámetros de la capa de Embedding

In [13]:
vocab_size = number_of_words
vocab_size

20000

In [14]:
embed_size = 128

## Paso 4: Construir la Red Neuronal Recurrente

### Definir el modelo

In [16]:
model = tf.keras.Sequential()

### Añadir la capa de embedding

se ocupa esta capa para pasar
 a un vector de palabras, para suministrar el texto en formato numerico, se ocupa para pasar el texto a una matriz donde cada fila es la valoracion y las columnas son las palabras
La columnas son la codificacion de las palabras
Conservar igual numeros de filas que las palabras y determina cuales palabras me interesa obtener

embed size se convierte las palbras a esa cantidad de neuoronas cada palabra se representa con 128 neuronas

128 columnas

In [17]:
model.add(tf.keras.layers.Embedding(vocab_size, embed_size, input_shape=(X_train.shape[1],)))

hacen que las neuronas voten la positividad o negatividad, despiertan una serie de neuronas

### Añadir la capa de LSTM

- unidades: 128
- función de activación: tanh

In [18]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

### Añadir la capa totalmente conectada de salida

- unidades: 1
- función de activación: sigmoid

prob entre 0 y 1, con la funcion sigmoid

In [19]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compilar el modelo

el optimizador rmsprod esta mas preparado para este tipo de redes neuronales recurrentes


In [20]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

100 palabras , de 128 de longitud en total dan un total de 256000 pesos para las redes neuronales de parametros a calcular

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


### Entrenar el modelo

In [27]:
model.fit(X_train, y_train, epochs=10, batch_size=200)

Epoch 1/10
125/125 [==============================] - 3s 28ms/step - loss: 0.0116 - accuracy: 0.9966
Epoch 2/10
125/125 [==============================] - 3s 26ms/step - loss: 0.0099 - accuracy: 0.9972
Epoch 3/10
125/125 [==============================] - 3s 27ms/step - loss: 0.0091 - accuracy: 0.9979
Epoch 4/10
125/125 [==============================] - 3s 27ms/step - loss: 0.0068 - accuracy: 0.9979
Epoch 5/10
125/125 [==============================] - 3s 27ms/step - loss: 0.0108 - accuracy: 0.9974
Epoch 6/10
125/125 [==============================] - 3s 27ms/step - loss: 0.0055 - accuracy: 0.9981
Epoch 7/10
125/125 [==============================] - 3s 26ms/step - loss: 0.0067 - accuracy: 0.9982
Epoch 8/10
125/125 [==============================] - 3s 27ms/step - loss: 0.0041 - accuracy: 0.9990
Epoch 9/10
125/125 [==============================] - 3s 27ms/step - loss: 0.0054 - accuracy: 0.9988
Epoch 10/10
125/125 [==============================] - 3s 27ms/step - loss: 0.0031 - accura

### Evaluar el modelo

In [28]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 1.4196 - accuracy: 0.8165


In [29]:
print("Test accuracy: {}".format(test_acurracy))

Test accuracy: 0.8165199756622314
